<a href="https://colab.research.google.com/github/myPar/NSU_Practice/blob/dev/baseline/RNNbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = Path('/content/drive/MyDrive/Colab Notebooks/data.txt')
df = pd.read_csv(filepath_or_buffer=file_path, sep=' ', header=0)

df.head

<bound method NDFrame.head of              Date      Time  qo_lc[m3/d]  qw_lc[m3/d]  qg_lc[m3/d]  \
0     07-May-2014  10:13:26      109.280    -0.312890       3810.3   
1     07-May-2014  10:14:26      113.460    -0.437350       3774.0   
2     07-May-2014  10:15:26      131.630    -0.533550       3892.5   
3     07-May-2014  10:16:26      140.670     0.160910       4043.1   
4     07-May-2014  10:17:26      150.020    -1.145600       4134.6   
...           ...       ...          ...          ...          ...   
1264  08-May-2014  07:34:35       98.744     0.310070       3174.5   
1265  08-May-2014  07:35:35       72.045     0.162760       3134.1   
1266  08-May-2014  07:36:35       87.370    -0.125770       3340.4   
1267  08-May-2014  07:37:35       32.121     0.021522       2834.1   
1268  08-May-2014  07:38:35       35.914     0.228020       2612.4   

      qo_sc[Sm3/d]  qw_sc[Sm3/d]  qg_sc[Sm3/d]  qo_scnp[Sm3/d]  \
0          108.430     -0.314870       92700.0         108.430 

In [4]:
print(df.columns)

Index(['Date', 'Time', 'qo_lc[m3/d]', 'qw_lc[m3/d]', 'qg_lc[m3/d]',
       'qo_sc[Sm3/d]', 'qw_sc[Sm3/d]', 'qg_sc[Sm3/d]', 'qo_scnp[Sm3/d]',
       'qw_scnp[Sm3/d]', 'qg_scnp[Sm3/d]', 'Fo_lc[%]', 'Fw_lc[%]', 'Fg_lc[%]',
       'WLR[%]', 'GVF[%]', 'GLR[m3/m3]', 'BSW[%]', 'GOR[Sm3/Sm3]',
       'GOR1[Sm3/Sm3]', 'mo_lc[kg/d]', 'mw_lc[kg/d]', 'mg_lc[kg/d]',
       'm_lc[kg/d]', 'mo_sc[kg/d]', 'mw_sc[kg/d]', 'mg_sc[kg/d]',
       'Mu_o_lc[cP]', 'Mu_l_lc[cP]', 'Do_lc[g/cm3]', 'Dw_lc[g/cm3]',
       'Dg_lc[g/cm3]', 'Dm_lc[g/cm3]', 'Dl_lc[g/cm3]', 'bo[Sm3/m3]',
       'bw[Sm3/m3]', 'bg[Sm3/m3]', 'Z', 'Rst[Sm3/Sm3]', 'Rwst[Sm3/Sm3]',
       'rgmp[Sm3/Sm3]', 'N32[cps]', 'N81[cps]', 'N356[cps]', 'NTotal[cps]',
       'DeadTime[s]', 'SampleTime[s]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]',
       'TAMB[DegC]', 'Dyn_DP[mbar]', 'OperatingPointLE[1/m]',
       'OperatingPointHE[1/m]', 'OilPointLE[1/m]', 'OilPointHE[1/m]',
       'WaterPointLE[1/m]', 'WaterPointHE[1/m]', 'GasPointLE[1/m]',
       'GasPoin

In [5]:
selected_rows = ['DPV[mbar]', 'PL[bara]', 'qg_sc[Sm3/d]', 'qo_sc[Sm3/d]', 'TL[DegC]']
outputs = ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']
inputs = np.setdiff1d(selected_rows, outputs)

df = df[selected_rows]

print(df.head)
print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

<bound method NDFrame.head of       DPV[mbar]  PL[bara]  qg_sc[Sm3/d]  qo_sc[Sm3/d]  TL[DegC]
0       1448.70    21.481       92700.0       108.430   -2.5452
1       1462.10    21.473       91895.0       112.580   -2.5323
2       1592.30    21.672       95860.0       130.550   -2.3931
3       1703.30    21.926      100690.0       139.440   -2.0451
4       1821.70    22.196      104290.0       148.640   -1.8009
...         ...       ...           ...           ...       ...
1264    1147.10    23.997       87402.0        97.703   -3.3542
1265    1090.50    23.919       85594.0        71.288   -3.2890
1266    1279.10    24.477       93660.0        86.386   -3.2191
1267     976.62    23.808       76564.0        31.789   -3.2819
1268        NaN       NaN       69609.0        35.567       NaN

[1269 rows x 5 columns]>
inputs: ['DPV[mbar]' 'PL[bara]' 'TL[DegC]']
outputs: ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']


In [6]:
# check categorical attributes
cardinality_hold = 0.15
expected_col_size = 1269

categorical_attributes = []

def check_attributes_info():
  for column in df.columns:
    col_data = df[column].to_numpy()
    size = len(col_data)
    
    assert size == expected_col_size
    cleared_data = col_data[~np.isnan(col_data)]
    
    card = len(np.unique(cleared_data))  # cardinality without Nan items
    skip_count = size - len(cleared_data)

    print(column + ": card=" + str(card) + "; skip: percent=" + str(skip_count * 100 / size) + ", count=" + str(skip_count))
    
    if card / size < cardinality_hold:
      categorical_attributes.append(card)

check_attributes_info()

if len(categorical_attributes) == 0:
  print("no categorical attributes")
else:
  print(categorical_attributes)

DPV[mbar]: card=1163; skip: percent=0.07880220646178093, count=1
PL[bara]: card=1042; skip: percent=0.07880220646178093, count=1
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
TL[DegC]: card=1214; skip: percent=0.07880220646178093, count=1
no categorical attributes


In [7]:
# supplement data
for column in df.columns:
  data =  np.array(df[column].to_numpy(), dtype=np.float64)

  for i in range(len(data)):
    if str(data[i]) == 'nan':
      data[i] = np.mean(data[~np.isnan(data)])
  df.drop([column], axis=1, inplace=True)
  df.insert(0, column, data)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
check_attributes_info()

TL[DegC]: card=1215; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
PL[bara]: card=1043; skip: percent=0.0, count=0
DPV[mbar]: card=1164; skip: percent=0.0, count=0


## Create model

In [9]:
# model parameters
batch_size = 60
timesteps = 60
input_size = len(inputs)
output_size = len(outputs)
lr = 0.001

mse = tf.keras.losses.MeanSquaredError()

model = tf.keras.Sequential()
model.add(Input((None, input_size)))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=120, activation='tanh', return_sequences=True))
model.add(SimpleRNN(units=2, activation='tanh', return_sequences=True))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 120)         14880     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 120)         28920     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 2)           246       
                                                                 
Total params: 44,046
Trainable params: 44,046
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss=mse, metrics=[mse], optimizer=Adam(learning_rate=lr))

## Prepare data

In [11]:
# normilize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function

result_interval = interval_th
normalized_df = df = ((df - df.min()) / (df.max() - df.min())) * (result_interval[1] - result_interval[0]) + result_interval[0]

train_df, test_df = train_test_split(normalized_df, test_size=0.3)
print(train_df.head)
print(test_df.head)

train_input_data = train_df[inputs].to_numpy()
train_output_data = train_df[outputs].to_numpy()

test_input_data = test_df[inputs].to_numpy()
test_output_data = test_df[outputs].to_numpy()

print("data shapes:")
print(train_input_data.shape)
print(test_input_data.shape)

print(train_output_data.shape)
print(test_output_data.shape)

<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
451   0.832516     -0.695149     -0.216606 -0.619961  -0.351240
341   0.580424      0.115008      0.253518 -0.257355   0.147548
83   -0.043586     -0.452693     -0.200432 -0.030726  -0.503746
267   0.334062     -0.486682     -0.543648 -0.915886  -0.588638
116   0.041701      0.092118      0.183057 -0.178034  -0.001602
...        ...           ...           ...       ...        ...
318   0.693085      0.381489      0.469744 -0.098714   0.447189
35   -0.142674     -0.916368     -0.929179 -0.925038  -0.780067
1243 -0.526883     -0.016324      0.122686  0.568533  -0.270877
463   0.460750      0.292700      0.056429 -0.428634   0.027087
1000 -0.209939     -0.347557      0.124527  0.443016  -0.268361

[888 rows x 5 columns]>
<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
720   0.296659     -0.831915     -0.329103 -0.137503  -0.492505
688   0.412111     

In [13]:
# formatting the data
def get_sequence(data):
  return np.array([data[i*timesteps : i*timesteps + timesteps] for i in range(data.shape[0] // timesteps)])

def get_remaining_data(data, sequences_count):
  return np.array(data[timesteps * sequences_count:])

def print_sequences_info(in_seq, out_seq, in_rem, out_rem):
  print(in_seq.shape)
  print(out_seq.shape)

  for i in range(len(in_seq)):
    print(str(i) + ": " + str(in_seq[i].shape))

  print("-----------")

  for i in range(len(out_seq)):
    print(str(i) + ": " + str(out_seq[i].shape))

  print("remaining data: ")
  print(in_rem.shape)
  print(out_rem.shape)

input_train_sequences = get_sequence(train_input_data)
output_train_sequences = get_sequence(train_output_data)

input_test_sequences = get_sequence(test_input_data)
output_test_sequences = get_sequence(test_output_data)

input_train_remaining = []
output_train_remaining = []

input_test_remaining = []
output_test_remaining = []

train_sequences_count = len(input_train_sequences)
assert train_sequences_count == len(output_train_sequences)

test_sequences_count = len(input_test_sequences)
assert test_sequences_count == len(output_test_sequences)

# get the remaining data from train samples:
if train_sequences_count * timesteps < train_input_data.shape[0]:
  input_train_remaining = get_remaining_data(train_input_data, train_sequences_count)

if train_sequences_count * timesteps < train_output_data.shape[0]:
  output_train_remaining = get_remaining_data(train_output_data, train_sequences_count)

# get the remaining data from test samples:
if test_sequences_count * timesteps < test_input_data.shape[0]:
  input_test_remaining = get_remaining_data(test_input_data, test_sequences_count)

if test_sequences_count * timesteps < test_output_data.shape[0]:
  output_test_remaining = get_remaining_data(test_output_data, test_sequences_count)

print("train smaples:")
print_sequences_info(input_train_sequences, output_train_sequences, input_train_remaining, output_train_remaining)
print("\n************************\n")
print("test samples:")
print_sequences_info(input_test_sequences, output_test_sequences, input_test_remaining, output_test_remaining)

train smaples:
(14, 60, 3)
(14, 60, 2)
0: (60, 3)
1: (60, 3)
2: (60, 3)
3: (60, 3)
4: (60, 3)
5: (60, 3)
6: (60, 3)
7: (60, 3)
8: (60, 3)
9: (60, 3)
10: (60, 3)
11: (60, 3)
12: (60, 3)
13: (60, 3)
-----------
0: (60, 2)
1: (60, 2)
2: (60, 2)
3: (60, 2)
4: (60, 2)
5: (60, 2)
6: (60, 2)
7: (60, 2)
8: (60, 2)
9: (60, 2)
10: (60, 2)
11: (60, 2)
12: (60, 2)
13: (60, 2)
remaining data: 
(48, 3)
(48, 2)

************************

test samples:
(6, 60, 3)
(6, 60, 2)
0: (60, 3)
1: (60, 3)
2: (60, 3)
3: (60, 3)
4: (60, 3)
5: (60, 3)
-----------
0: (60, 2)
1: (60, 2)
2: (60, 2)
3: (60, 2)
4: (60, 2)
5: (60, 2)
remaining data: 
(21, 3)
(21, 2)


In [16]:
def add_remaining(data, remaining):
  data = list(data)
  data.append(remaining)
  return data

input_train_tensor = add_remaining(input_train_sequences, input_train_remaining)
output_train_tensor = add_remaining(output_train_sequences, output_train_remaining)

input_test_tensor = add_remaining(input_test_sequences, input_test_remaining)
output_test_tensor = add_remaining(output_test_sequences, output_test_remaining)

print(len(input_train_tensor))
print(len(output_train_tensor))

def print_batches_shapes(in_tensor, out_tensor):
  print(" input batches shapes:")

  for i in range(len(in_tensor)):
    print(" " + str(in_tensor[i].shape))

  print(" ------------")
  print(" output batches shapes:")

  for i in range(len(out_tensor)):
    print(" " + str(out_tensor[i].shape))

print("train tensors:")
print_batches_shapes(input_train_tensor, output_train_tensor)
print()
print("test tensors:")
print_batches_shapes(input_test_tensor, output_test_tensor)

15
15
train tensors:
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (48, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (48, 2)

test tensors:
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (21, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (21, 2)


## Train model

In [17]:
epoch_count = 70
batch_count = len(input_train_tensor)

input_shape = (1, timesteps, input_size)
output_shape = (1, timesteps, output_size)

input_shape_remaining = (1, input_train_remaining.shape[0], input_train_remaining.shape[1])
output_shape_remaining = (1, output_train_remaining.shape[0], output_train_remaining.shape[1])

#print("remainig shapes: " + str(input_shape_remaining) + ", " + str(output_shape_remaining))

for epoch in range(epoch_count):
  for batch_idx in range(batch_count):
    in_shape = input_shape
    out_shape = output_shape

    if batch_idx == batch_count - 1:
      in_shape = input_shape_remaining 
      out_shape = output_shape_remaining

    input_batch = input_train_tensor[batch_idx].reshape(in_shape)
    output_batch = output_train_tensor[batch_idx].reshape(out_shape)

    loss = model.train_on_batch(input_batch, output_batch, reset_metrics=False, return_dict=True)
  print("epoch-" + str(epoch) + ";" + str(loss))

epoch-0;{'loss': 0.17980502545833588, 'mean_squared_error': 0.17980502545833588}
epoch-1;{'loss': 0.1177729070186615, 'mean_squared_error': 0.1177729070186615}
epoch-2;{'loss': 0.09080991894006729, 'mean_squared_error': 0.09080991894006729}
epoch-3;{'loss': 0.07562922686338425, 'mean_squared_error': 0.07562922686338425}
epoch-4;{'loss': 0.06518927216529846, 'mean_squared_error': 0.06518927216529846}
epoch-5;{'loss': 0.05787305906414986, 'mean_squared_error': 0.05787305906414986}
epoch-6;{'loss': 0.05247960612177849, 'mean_squared_error': 0.05247960612177849}
epoch-7;{'loss': 0.04831886291503906, 'mean_squared_error': 0.04831886291503906}
epoch-8;{'loss': 0.044980209320783615, 'mean_squared_error': 0.044980209320783615}
epoch-9;{'loss': 0.042215943336486816, 'mean_squared_error': 0.042215943336486816}
epoch-10;{'loss': 0.039886005222797394, 'mean_squared_error': 0.039886005222797394}
epoch-11;{'loss': 0.037899646908044815, 'mean_squared_error': 0.037899646908044815}
epoch-12;{'loss': 0.

## Test model

In [19]:
# test on batches
batch_count = len(input_test_tensor)

input_shape = (1, timesteps, input_size)
output_shape = (1, timesteps, output_size)

input_shape_remaining = (1, input_test_remaining.shape[0], input_test_remaining.shape[1])
output_shape_remaining = (1, output_test_remaining.shape[0], output_test_remaining.shape[1])

loss = 0

for batch_idx in range(batch_count):
  in_shape = input_shape
  out_shape = output_shape

  if batch_idx == batch_count - 1:
    in_shape = input_shape_remaining 
    out_shape = output_shape_remaining

  input_batch = input_test_tensor[batch_idx].reshape(in_shape)
  output_batch = output_test_tensor[batch_idx].reshape(out_shape)

  loss = model.test_on_batch(input_batch, output_batch, return_dict=True, reset_metrics=False)

print("test loss=" + str(loss))

test loss={'loss': 0.014609351754188538, 'mean_squared_error': 0.014609351754188538}
